*Note: Data Used is `Dummy` don't try to drive insights*

# Mixed Linear Model (MixedLM) Example

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 2(a). Load and Prepare Data Sales

In [ ]:
df = pd.read_csv('/content/Example of Sales Longitudinal Data.csv')
df.head()

,Pharmacy Code,01/01/2025,01/02/2025,01/03/2025,01/04/2025,01/05/2025,01/06/2025,01/07/2025,01/08/2025,01/09/2025,...,01/03/2026,01/04/2026,01/05/2026,01/06/2026,01/07/2026,01/08/2026,01/09/2026,01/10/2026,01/11/2026,01/12/2026
0,PH001,319,523,257,80,975,997,38,257,451,...,857,736,447,813,245,463,33,1022,463,818
1,PH002,959,231,557,226,235,345,772,235,143,...,474,597,567,751,599,739,938,375,555,85
2,PH003,616,555,236,591,776,358,59,390,951,...,516,186,465,185,15,76,1100,1099,999,423
3,PH004,256,62,490,1075,625,989,972,491,662,...,951,565,338,547,698,921,682,877,521,652
4,PH005,989,284,365,792,432,381,308,919,1070,...,602,1098,432,253,525,1041,210,299,1083,34


## 3(a). Data Preprocessing
- converted the data to time-series.

In [ ]:
# Reshape the data using melt
df_melted = pd.melt(df, id_vars=['Pharmacy Code'], var_name='Date', value_name='Sales')

# Convert 'Date' column to datetime objects
df_melted['Date'] = pd.to_datetime(df_melted['Date'])

# Set 'Date' as index for time-series operations
df_ts = df_melted.set_index('Date')

# Display the resulting time series data
df_ts.head()

,Pharmacy Code,Sales
Date,,
2025-01-01,PH001,319
2025-01-01,PH002,959
2025-01-01,PH003,616
2025-01-01,PH004,256
2025-01-01,PH005,989


In [ ]:
df_ts.sort_values(by=['Pharmacy Code','Date'], inplace=True)
df_ts.head()

,Pharmacy Code,Sales
Date,,
2025-01-01,PH001,319
2025-01-02,PH001,523
2025-01-03,PH001,257
2025-01-04,PH001,80
2025-01-05,PH001,975


## 2(b). Load and Prepare Promotion Data

In [ ]:
promotion = pd.read_csv('/content/Promotion Data by Pharmacy.csv')
promotion.head()

,Pharmacy Code,01/01/2025,01/02/2025,01/03/2025,01/04/2025,01/05/2025,01/06/2025,01/07/2025,01/08/2025,01/09/2025,...,01/03/2026,01/04/2026,01/05/2026,01/06/2026,01/07/2026,01/08/2026,01/09/2026,01/10/2026,01/11/2026,01/12/2026
0,PH001,11,1,5,9,10,7,6,12,6,...,7,11,5,2,0,8,0,6,1,6
1,PH002,5,10,3,2,8,2,5,3,7,...,4,12,7,3,6,1,5,11,3,0
2,PH003,9,9,8,6,4,6,1,6,0,...,9,1,10,7,8,7,0,11,10,9
3,PH004,5,7,8,12,1,6,10,3,4,...,0,10,10,1,7,11,6,7,4,3
4,PH005,12,7,0,3,7,6,8,3,3,...,8,5,6,11,7,10,9,11,4,1


## 3(b). Data Preprocessing
- converted the data to time-series.

In [ ]:
# Melt the promotion DataFrame
promotion_melted = pd.melt(promotion, id_vars=['Pharmacy Code'], var_name='Date', value_name='Promotion')

# Convert 'Date' column to datetime objects
promotion_melted['Date'] = pd.to_datetime(promotion_melted['Date'])

# Sort the melted DataFrame by 'Pharmacy Code' and 'Date'
promotion_melted.sort_values(by=['Pharmacy Code', 'Date'], inplace=True)

# Display the sorted and melted DataFrame
promotion_melted


,Pharmacy Code,Date,Promotion
0,PH001,2025-01-01,11
60,PH001,2025-01-02,1
120,PH001,2025-01-03,5
180,PH001,2025-01-04,9
240,PH001,2025-01-05,10
...,...,...,...
1199,PH060,2026-01-08,1
1259,PH060,2026-01-09,11
1319,PH060,2026-01-10,5
1379,PH060,2026-01-11,10


In [ ]:
# Set 'Date' as index for time-series operations
promotion_ts = promotion_melted.set_index('Date')

# Display the resulting time series data
promotion_ts


,Pharmacy Code,Promotion
Date,,
2025-01-01,PH001,11
2025-01-02,PH001,1
2025-01-03,PH001,5
2025-01-04,PH001,9
2025-01-05,PH001,10
...,...,...
2026-01-08,PH060,1
2026-01-09,PH060,11
2026-01-10,PH060,5


## 4. Feature Engineering

In [ ]:
merged_df = pd.merge(df_ts.reset_index(), promotion_ts.reset_index(), on=['Pharmacy Code', 'Date'], how='left')
merged_df.set_index('Date', inplace=True)
merged_df.head()


,Pharmacy Code,Sales,Promotion
Date,,,
2025-01-01,PH001,319,11
2025-01-02,PH001,523,1
2025-01-03,PH001,257,5
2025-01-04,PH001,80,9
2025-01-05,PH001,975,10


In [ ]:
merged_df

,Pharmacy Code,Sales,Promotion
Date,,,
2025-01-01,PH001,319,11
2025-01-02,PH001,523,1
2025-01-03,PH001,257,5
2025-01-04,PH001,80,9
2025-01-05,PH001,975,10
...,...,...,...
2026-01-08,PH060,376,1
2026-01-09,PH060,728,11
2026-01-10,PH060,666,5


In [ ]:
# Fill missing values carefully when shifts are not available
merged_df['Adstock'] = (
    merged_df['Promotion'] +
    0.5 * merged_df['Promotion'].shift(1).fillna(0) +
    0.25 * merged_df['Promotion'].shift(2).fillna(0)
)

# To adjust the weighting when lag values are missing
def compute_adstock(row):
    if pd.notna(row['Promotion_shift2']) and pd.notna(row['Promotion_shift1']):
        return row['Promotion'] + 0.5 * row['Promotion_shift1'] + 0.25 * row['Promotion_shift2']
    elif pd.notna(row['Promotion_shift1']):
        return row['Promotion'] + 0.5 * row['Promotion_shift1']
    else:
        return row['Promotion']

# Create shifted columns
merged_df['Promotion_shift1'] = merged_df['Promotion'].shift(1)
merged_df['Promotion_shift2'] = merged_df['Promotion'].shift(2)

# Apply row-wise computation
merged_df['Adstock'] = merged_df.apply(compute_adstock, axis=1)

# Drop helper columns
merged_df.drop(['Promotion_shift1', 'Promotion_shift2'], axis=1, inplace=True)

merged_df.head()


,Pharmacy Code,Sales,Promotion,Adstock
Date,,,,
2025-01-01,PH001,319,11,11.00
2025-01-02,PH001,523,1,6.50
2025-01-03,PH001,257,5,8.25
2025-01-04,PH001,80,9,11.75
2025-01-05,PH001,975,10,15.75


In [ ]:
merged_df

,Pharmacy Code,Sales,Promotion,Adstock
Date,,,,
2025-01-01,PH001,319,11,11.00
2025-01-02,PH001,523,1,6.50
2025-01-03,PH001,257,5,8.25
2025-01-04,PH001,80,9,11.75
2025-01-05,PH001,975,10,15.75
...,...,...,...,...
2026-01-08,PH060,376,1,2.50
2026-01-09,PH060,728,11,12.00
2026-01-10,PH060,666,5,10.75


In [ ]:
merged_df['lag1'] = merged_df.groupby('Pharmacy Code')['Sales'].shift(1)
merged_df['lag2'] = merged_df.groupby('Pharmacy Code')['Sales'].shift(2)
merged_df

,Pharmacy Code,Sales,Promotion,Adstock,lag1,lag2
Date,,,,,,
2025-01-01,PH001,319,11,11.00,NaN,NaN
2025-01-02,PH001,523,1,6.50,319.0,NaN
2025-01-03,PH001,257,5,8.25,523.0,319.0
2025-01-04,PH001,80,9,11.75,257.0,523.0
2025-01-05,PH001,975,10,15.75,80.0,257.0
...,...,...,...,...,...,...
2026-01-08,PH060,376,1,2.50,642.0,1081.0
2026-01-09,PH060,728,11,12.00,376.0,642.0
2026-01-10,PH060,666,5,10.75,728.0,376.0


In [ ]:
merged_df.fillna(0, inplace=True)
merged_df

,Pharmacy Code,Sales,Promotion,Adstock,lag1,lag2
Date,,,,,,
2025-01-01,PH001,319,11,11.00,0.0,0.0
2025-01-02,PH001,523,1,6.50,319.0,0.0
2025-01-03,PH001,257,5,8.25,523.0,319.0
2025-01-04,PH001,80,9,11.75,257.0,523.0
2025-01-05,PH001,975,10,15.75,80.0,257.0
...,...,...,...,...,...,...
2026-01-08,PH060,376,1,2.50,642.0,1081.0
2026-01-09,PH060,728,11,12.00,376.0,642.0
2026-01-10,PH060,666,5,10.75,728.0,376.0


## 5. Setup for MixedLM Model

In [ ]:
import statsmodels.formula.api as smf

# Define and fit the Linear Mixed Model
model = smf.mixedlm("Sales ~ Adstock + lag1 + lag2",
        merged_df, groups=merged_df["Pharmacy Code"], re_formula="~Adstock")

## 6. Fit the MixedLM Model

In [ ]:
result = model.fit()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "

## 7. Model Diagnostics and Summary

In [ ]:
print(result.summary())

               Mixed Linear Model Regression Results
Model:                MixedLM    Dependent Variable:    Sales      
No. Observations:     1440       Method:                REML       
No. Groups:           60         Scale:                 93453.7578 
Min. group size:      24         Log-Likelihood:        -10306.5164
Max. group size:      24         Converged:             No         
Mean group size:      24.0                                         
-------------------------------------------------------------------
                      Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
-------------------------------------------------------------------
Intercept             619.987   33.942 18.266 0.000 553.462 686.513
Adstock                -1.755    2.147 -0.817 0.414  -5.962   2.453
lag1                   -0.021    0.025 -0.830 0.406  -0.070   0.028
lag2                   -0.059    0.025 -2.337 0.019  -0.108  -0.009
Group Var           18391.537                                  

### Random Effect Coefficient

In [ ]:
random_effect = result.random_effects
random_effect

{'PH001': Group     -98.953549
 Adstock     5.130581
 dtype: float64,
 'PH002': Group     -72.391181
 Adstock     3.655912
 dtype: float64,
 'PH003': Group     -112.313285
 Adstock      5.716648
 dtype: float64,
 'PH004': Group      74.838194
 Adstock    -3.804048
 dtype: float64,
 'PH005': Group     -16.670238
 Adstock     0.847355
 dtype: float64,
 'PH006': Group     -52.829517
 Adstock     2.740386
 dtype: float64,
 'PH007': Group     -166.239165
 Adstock      8.506321
 dtype: float64,
 'PH008': Group      60.618875
 Adstock    -3.116549
 dtype: float64,
 'PH009': Group      31.893064
 Adstock    -1.655307
 dtype: float64,
 'PH010': Group     -21.775292
 Adstock     1.120633
 dtype: float64,
 'PH011': Group      45.760483
 Adstock    -2.280489
 dtype: float64,
 'PH012': Group      25.598083
 Adstock    -1.255848
 dtype: float64,
 'PH013': Group     -30.671385
 Adstock     1.620605
 dtype: float64,
 'PH014': Group      72.519314
 Adstock    -3.701315
 dtype: float64,
 'PH015': Group 

## 8. Model Prediction

In [ ]:
# 1. Fixed effect prediction
fixed_preds = result.predict(merged_df)

# 2. Get the random effects learned during training
random_effects = result.random_effects  # Dict: {group_id: value}

# 3. Map the group-level random effect to the rows in the data
merged_df['random_effect'] = merged_df['Pharmacy Code'].map(
    lambda x: random_effects.get(x, {'Group': 0}).get('Group', 0)
)

# 4. Final prediction = fixed + random
full_preds = fixed_preds + merged_df['random_effect']

# now calculate r2 score using sklearn
from sklearn.metrics import r2_score
r2 = r2_score(merged_df['Sales'], full_preds)
print(f"R2 Score: {r2}")

R2 Score: 0.059101299170421506


In [ ]:
# Calculate R-squared using fittedvalues
r2_fitted = r2_score(merged_df['Sales'], result.fittedvalues)
print(f"R2 Score (using fittedvalues): {r2_fitted}")

R2 Score (using fittedvalues): 0.054121324758319767
